<a href="https://colab.research.google.com/github/Shamvv/Cyber-Security/blob/main/vulnerability_detection_on_code_snippets.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Prerequisites

In [1]:
!pip install transformers torch datasets

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 472.7/472.7 kB 8.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 4.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 6.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 10.1 MB/s eta 0:00:00


Load the Dataset

In [9]:
!pip install datasets
from datasets import load_dataset, ClassLabel, Value

from datasets import load_dataset

dataset = load_dataset("code_search_net", "python")

# Check the actual keys in the dataset:
print(dataset["train"][0].keys())  # Print keys of the first item in the 'train' split

# Assuming the keys are "whole_func_string" for code and "func_documentation_string" for docstrings:
# Before mapping, check if "func_documentation_string" has unique values to determine the number of labels.
# Get unique labels from all splits (train, validation, test if available)
all_docstrings = []
for split in dataset.keys():
    all_docstrings.extend(dataset[split]["func_documentation_string"])
unique_labels = set(all_docstrings)

# Create a ClassLabel feature with the unique labels
label_feature = ClassLabel(num_classes=len(unique_labels), names=list(unique_labels))

dataset = dataset.map(lambda x: {"text": x["whole_func_string"], "label": label_feature.str2int(x["func_documentation_string"])})

dict_keys(['repository_name', 'func_path_in_repository', 'func_name', 'whole_func_string', 'language', 'func_code_string', 'func_code_tokens', 'func_documentation_string', 'func_documentation_tokens', 'split_name', 'func_code_url'])


Map:   0%|          | 0/412178 [00:00<?, ? examples/s]

Map:   0%|          | 0/22176 [00:00<?, ? examples/s]

Map:   0%|          | 0/23107 [00:00<?, ? examples/s]

Preprocessing

In [ ]:
tokenizer = AutoTokenizer.from_pretrained("microsoft/codebert-base")

# Tokenize the dataset
def tokenize_function(example):
    return tokenizer(
        example["text"],
        padding="max_length",
        truncation=True,
        max_length=256,
    )

tokenized_datasets = dataset.map(tokenize_function, batched=True)


/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


Map:   0%|          | 0/412178 [00:00<?, ? examples/s]

Load and Fine-tune the Model

In [7]:
from transformers import AutoModelForSequenceClassification, TrainingArguments, Trainer
import torch

# Load the pre-trained model
model = AutoModelForSequenceClassification.from_pretrained("microsoft/codebert-base", num_labels=2)

# Split the dataset into train and test
# Apply train_test_split to the 'train' split within the DatasetDict and update the 'train' and 'test' splits
splits = tokenized_datasets["train"].train_test_split(test_size=0.1)
tokenized_datasets["train"] = splits["train"]
tokenized_datasets["test"] = splits["test"]

# Set up training arguments
training_args = TrainingArguments(
    output_dir="./results",
    evaluation_strategy="epoch",
    learning_rate=2e-5,
    per_device_train_batch_size=8,
    per_device_eval_batch_size=8,
    num_train_epochs=3,
    weight_decay=0.01,
)

# Create Trainer
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_datasets["train"],
    eval_dataset=tokenized_datasets["test"],
)

# Train the model
trainer.train()

Some weights of RobertaForSequenceClassification were not initialized from the model checkpoint at microsoft/codebert-base and are newly initialized: ['classifier.dense.bias', 'classifier.dense.weight', 'classifier.out_proj.bias', 'classifier.out_proj.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
/usr/local/lib/python3.10/dist-packages/transformers/training_args.py:1525: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(
wandb: WARNING The `run_name` is currently set to the same value as `TrainingArguments.output_dir`. If this was not intended, please specify a different run name by setting the `TrainingArguments.run_name` parameter.
wandb: Using wandb-core as the SDK backend. Please refer to https://wandb.me/wandb-core for more information.


<IPython.core.display.Javascript object>

wandb: Logging into wandb.ai. (Learn how to deploy a W&B server locally: https://wandb.me/wandb-server)
wandb: You can find your API key in your browser here: https://wandb.ai/authorize
wandb: Paste an API key from your profile and hit enter, or press ctrl+c to quit:

 ··········


wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc


ValueError: too many dimensions 'str'

Evaluation

In [ ]:
from sklearn.metrics import classification_report

# Get predictions
predictions = trainer.predict(tokenized_datasets["test"])
preds = torch.argmax(predictions.predictions, axis=-1)

# Evaluate
print(classification_report(predictions.label_ids, preds))


Inference on New Code Snippets

In [ ]:
def classify_code_snippet(code_snippet):
    inputs = tokenizer(code_snippet, return_tensors="pt", padding="max_length", truncation=True, max_length=256)
    outputs = model(**inputs)
    predictions = torch.argmax(outputs.logits, dim=1)
    return "Vulnerable" if predictions[0].item() == 1 else "Non-vulnerable"

# Example usage
code_snippet = "def vulnerable_code():\n    eval(input())"
print(classify_code_snippet(code_snippet))
